<a href="https://colab.research.google.com/github/MicroprocessorX069/Basic-LSTM-Types/blob/master/Basic_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

print("Tensorflow version: ",tf.__version__)

In [2]:
import os
path=os.getcwd()

text=open(path+'/Iliad_v3.txt','rb').read().decode(encoding='utf-8')
print("Text is {} characters long".format(len(text)))

Text is 886809 characters long


In [3]:
words=[w for w in text.split(' ') if w.strip()!='' or w=='\n']
print("Text is {} words long".format(len(words)))

Text is 153260 words long


In [0]:
vocab=sorted(set(text))
print("There are {} unique characters".format(len(vocab)))
char2int={c:i for i,c in enumerate(vocab)}
int2char=np.array(vocab)
print("Vector:\n")
for char,_ in zip(char2int,range(len(vocab))):
  print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

In [0]:
text_as_int=np.array([char2int[ch] for ch in text],dtype=np.int32)
print('{}\n mapped to integers:\n {}'.format(repr(text[:100])))